# Context

## Quick reminder on last course

XX

## Goal of this course

XX


# Concepts

## Model Evaluation

For regressors, the evaluation of the model is less straightforward and id usually based upon the average residual between the actual data and the model predictions. __Root mean squared error__ (RMSE) and __mean absolute error__ (MAE) are two commonly used examples of these. Once we have a representation of the error produced by the model we can compare that error with the error of the most simple model of the data (usually the mean). This statistic is known as the coefficient of determination or __R-squared__ and represents the part of variability the model managed to get.

# Regression

The first thing to do is to get our data back.

In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
# We put both target arrays (regression and classification) in the same txt file
# As both target arrays have the same size we just need to split it it two
# and get the correct part for the prediction task
target = np.split(np.loadtxt('target.txt'), 2)[0].flatten()
features = pd.read_csv('features.csv')

# User id as index
features = features.set_index('user_id')

In [3]:
features.head()

,tw_lag7_watched,tw_lag6_watched,tw_lag5_watched,tw_lag4_watched,tw_lag3_watched,tw_lag2_watched,tw_lag1_watched,average_completion,total_sessions,total_watched,...,most_weekday_weekday_1,most_weekday_weekday_2,most_weekday_weekday_3,most_weekday_weekday_4,most_weekday_weekday_5,most_weekday_weekday_6,most_timeday_Afternoon,most_timeday_Evening,most_timeday_Morning,most_timeday_Night
user_id,,,,,,,,,,,,,,,,,,,,,
0001c6,16.679200,0.000000,0.0000,0.000000,0.000000,0.152550,0.000000,0.371496,2,16.831750,...,1,0,0,0,0,0,0,1,0,0
000c1a,0.162867,0.147467,107.0984,145.686233,2.286283,100.487767,132.432083,0.233136,28,488.301100,...,0,0,1,0,0,0,0,0,1,0
001c53,1.866300,0.000000,0.0000,0.000000,1.309867,0.000000,0.000000,0.489419,3,3.176167,...,0,1,0,0,0,0,0,0,1,0
001d44,0.000000,0.000000,0.0000,14.547700,0.000000,0.000000,0.248017,0.058203,2,14.795717,...,0,0,0,0,0,1,0,0,1,0
002b2e,291.477033,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.228233,17,291.477033,...,0,1,0,0,0,0,0,1,0,0


In [4]:
target[:10]

array([  1.44833333e-01,   3.18047633e+02,   1.98035000e+00,
         1.00590667e+01,   0.00000000e+00,   4.79261667e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00])

## Baseline forecasts

As mentionned before we should have a baseline to compare the performance of our models with. Here as we are forecasting a quantitative variable, a simple model is for example the one that predicts a constant value like the mean or the median we can observe on our past data. 

Note as well that by default `scikit` maximises metrics during gridsearch etc. Error metrics have been implemented as a negative number within the scoring function even if that is different to the usual. We therefore negate the baseline numbers to make them comparable and look for those models that maximise this number.

In [5]:
# Create a baseline to compare our results to (mean and median minutes watched and 0)
mean=np.mean(target)
median=np.median(target)

mean_forecast=[mean]*len(target)
median_forecast=[median]*len(target)
zero_forecast=[0]*len(target)

In [7]:
print("Mean: "+str(mean))
print("Median: "+str(median))

Mean: 75.9168097123
Median: 0.0


We have lots of 0s in our target, so lots of user who don't come back on iPlayer for the last two weeks.

In [6]:
# Compute the errors for these different baseline
from sklearn import metrics
print("Score if we forecast the mean:",
      -metrics.mean_absolute_error(target,mean_forecast))
print("Score if we forecast the median:",
      -metrics.mean_absolute_error(target,median_forecast))
print("Score if we forecast zero:",
      -metrics.mean_absolute_error(target,zero_forecast))

Score if we forecast the mean: -115.319756974
Score if we forecast the median: -75.9168097123
Score if we forecast zero: -75.9168097123
